# Traditional Word Embedding

In [1]:
# import packages
from gensim.models import word2vec
from monpa import utils
from tqdm import tqdm
import codecs
import csv
import logging
import monpa
import numpy as np
import os
import pandas as pd
import spacy

os.getcwd()

+---------------------------------------------------------------------+
  Welcome to MONPA: Multi-Objective NER POS Annotator for Chinese
+---------------------------------------------------------------------+
已找到 model檔。Found model file.


'D:\\Windows_Storage\\Storage\\Github\\KnowledgeGraph\\scripts'

# Configurations

In [2]:
selected_ratio = 0.5
data_entity = pd.read_csv("../results/210408_result/210408_dataset_entity_result_MONPA.csv", sep=",", encoding="utf8")
data_relation = pd.read_csv("../results/210408_result/210408_dataset_relation_result_MONPA.csv", sep=",", encoding="utf8")
output_token_for_word2vec = "../results/210408_result/210408_token_sentences_for_word2vec.csv"
model_save_path = "../models/210408_word2vec.model"

In [3]:
# MONPA
relation_dependencies_possible_list = []
relation_pos_possible_list = ["VH", "VC", "VJ", "VA"]
relation_pos_re = "^[V]"
entity_dependencies_possible_list = []
entity_pos_possible_list = ["Na", "Nv", "Neu", "Nes", "Nf", "Ng", "Nh", "Neqa", "Nep", "Ncd", "FW", "DE"]
entity_pos_re = "^[N]"
splitter_pos_list = ["COMMACATEGORY", "PERIODCATEGORY"]

# spaCy
# relation_dependencies_possible_list = ["ROOT", "nmod:prep", "prep", "agent", ]
# relation_pos_possible_list = ["VERB"]
# relation_pos_re = "^[V]"
# entity_dependencies_possible_list = ["compound:nn", "nsubj", "dep", "dobj"]
# entity_pos_possible_list = ["NOUN", "PROPN"]
# entity_pos_re = "^[N]"
# bracket_pos_list = []

# Common, usually entity
sentences_splitter = ["，", "。", "！", "!", "？", "?", "；", ";", "."] + ["\r\n" * i for i in range(0, 100)]
bracket_entity_list_first = ["(", "（", "[", "［", "{", "｛", "<", "＜", "〔", "【", "〖", "《", "〈"]
bracket_entity_list_last = [")", "）", "]", "］", "}", "｝", ">", "＞", "〕", "】", "〗", "》", "〉"]
punct_entity_list = [" " * i for i in range(0, 100)]
conjuction_entity_list = ["的", "、", "之", "及", "與"]
not_entity_relation_list = ["的", "、", "之", "及", "與", "\r\n \r\n ", "\r\n \r\n  "] +\
[" " * i for i in range(0, 100)] + ["\n" * i for i in range(0, 100)]

# Import stopwords list

In [4]:
stopword_list = []

for fileIndex, fileElement in enumerate(os.listdir("./stopwords/")):
    if fileElement[-2:] != "md":
        data_temp = pd.read_csv("./stopwords/" + fileElement, encoding="utf8", sep="@#$%&*")
        stopword_list += data_temp.iloc[:, 0].tolist()
        
stopword_list = list(dict.fromkeys(stopword_list))
        
print(len(stopword_list), "\n\n", stopword_list[-50:])     


3249 

 ['where', 'whereafter', 'whereas', 'whereby', 'wherein', "where's", 'whereupon', 'wherever', 'whether', 'which', 'while', 'whither', 'who', 'whoever', 'whole', 'whom', "who's", 'whose', 'why', 'will', 'willing', 'wish', 'with', 'within', 'without', 'wonder', "won't", 'would', "wouldn't", 'yes', 'yet', 'you', "you'd", "you'll", 'your', "you're", 'yours', 'yourself', 'yourselves', "you've", 'zero', 'zt', 'ZT', 'zz', 'ZZ', '準備', '覆雜', '心裏', '註意', '裏面']


<ipython-input-4-ecedad700bed>:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data_temp = pd.read_csv("./stopwords/" + fileElement, encoding="utf8", sep="@#$%&*")


# Create User-Defined Dict by entities

In [5]:
print(int(np.around(len(data_entity) * selected_ratio, decimals=0)), int(np.around(len(data_relation) * selected_ratio, decimals=0)))
data_dict = pd.concat([data_entity.iloc[:int(np.around(len(data_entity) * selected_ratio, decimals=0)),:],\
                    data_relation.iloc[:int(np.around(len(data_relation) * selected_ratio, decimals=0)),:]], ignore_index=True)
print(len(data_dict))

entity_list = data_dict.iloc[:, 0].tolist()

print(entity_list[:10], entity_list[-10:])

# remove blank inside entity element
for elementIndex, element in enumerate(entity_list):
    entity_list[elementIndex] = str(entity_list[elementIndex]).replace(" ", "")
    entity_list[elementIndex] = str(entity_list[elementIndex]).replace("  ", "")
    entity_list[elementIndex] = str(entity_list[elementIndex]).replace("\"", "")

for i in range(len(entity_list)):
    if type(data_dict.loc[i, "Ratio"]) == pd.Series:
        print(data_dict.loc[i, "Ratio"])
    
    
data_dict_output = pd.DataFrame({
    "0": entity_list,
    "1": [int(1000000000 * data_dict.loc[i, "Ratio"]) for i in range(len(entity_list))],
    "2": ["UserEntity" for i in range(len(entity_list))],
})

data_dict_output.to_csv("./dicts/monpa_entity_dict.txt", encoding="utf8", sep=" ", index=None, header=None)

4462 1404
5866
['晶圓', '技術', '製程', '晶片', '產品', '設備', '表面', '資源', '成本', '測品'] ['設計業', '連絡', '進 ', '進出', '進化', '設立', '進流水', '設法', '設有', '設施']


# Import and Segment data 

In [6]:
textElement = ""

for fileIndex, fileElement in enumerate(tqdm(os.listdir("../data/"))):
    file = codecs.open("../data/" + fileElement, 'r', encoding='utf8', errors='ignore')
    for textIndex, textLines in enumerate(file):
        textElement += textLines
        
    
print("Text Length: ", len(textElement), " \n\n", textElement[-100:])

100%|██████████| 15/15 [00:00<00:00, 245.91it/s]

Text Length:  207715  

 管中流過，凝結下來之蒸

氣附著管外殼上。蒸氣可在管外凝結成一層薄膜後，再排

至儲存槽，或排出後做適當之處置。在接觸式冷凝器中，

則噴灑冷的液體以冷凝廢氣中之揮發性成份。 

5 -30 





In [7]:
# read in entity dictionary

monpa.load_userdict("./dicts/monpa_entity_dict.txt")

In [8]:
# create splitted tect lit by monpa

textList = utils.short_sentence(textElement)

In [9]:
# divide sentences and segment by monpa
cutResultList = []


for textIndex, textElement in enumerate(tqdm(textList)):
    cutElements = []
    cutPOSs = []
    # pseg function
    cutResult = monpa.pseg(textElement)
    # append entity and pos seperately
    for cutIndex, cutElement in enumerate(cutResult):
        cutElements.append(cutElement[0])
        cutPOSs.append(cutElement[1])
    # eliminate entity between two brackets
    for elementIndex, element in enumerate(cutElements):
        # delete elements that's between two bracket
        if cutElements[elementIndex] in bracket_entity_list_first:
            left_bracket_index = bracket_entity_list_first.index(cutElements[elementIndex])
            findingLimit =  (elementIndex+11) if (elementIndex+11) <= (len(cutElements)) else len(cutElements)
            for findingLeftIndex in range(elementIndex+1, findingLimit):
                if cutElements[findingLeftIndex] == bracket_entity_list_last[left_bracket_index]:
                    for removalIndex in range(elementIndex, findingLeftIndex+1):
                        cutElements[removalIndex] = ""
                    break
        elif element in punct_entity_list:
            cutElements[elementIndex] = ""
        elif cutResult[elementIndex][1] in splitter_pos_list or cutResult[elementIndex][0] in sentences_splitter:
            # appending subsentences
            cutElements[elementIndex] = ""
            subCutElements = cutElements[:elementIndex]
            subCutElements =  list(filter(("").__ne__, subCutElements))
            cutResultList.append(subCutElements)
            for appendingIndex in range(0, elementIndex):
                cutElements[appendingIndex] = ""
    cutElements = list(filter(("").__ne__, cutElements))
    cutResultList.append(cutElements)

print(cutResultList[:5])

100%|██████████| 1332/1332 [12:43<00:00,  1.74it/s]

[['安森美', '半導體'], ['今天', '發表', '新', '的', 'R', 'D', 'M', '系列', '矽', '光電', '倍增', '管', '陣列'], ['將', '光學', '雷達', '感測器', '能力', '擴', '展', '到', '其', '廣泛', '的', '智慧', '感測', '方案', '陣容'], ['ArrayR', 'D', 'M', '-', '0112', 'A', '20', '-', 'QFN', '是', '市場', '上', '首款', '符合', '車規', '的', 'SiP', 'M', '產品'], ['以', '滿足', '汽車', '產業', '及', '其他', '領域', 'Li', 'D', 'AR', '應', '用', '中', '不斷', '增長', '的', '需求']]


In [10]:
# create data for word2vec training

lineList = []

for sentencesIndex, sentencesElementList in enumerate(tqdm(cutResultList)):
    tokenElement = ""
    for elementIndex, element in enumerate(sentencesElementList):
        if element not in stopword_list:
            tokenElement += (element + " ")
   
    lineList.append(tokenElement)
    
data_output_word2vec = pd.DataFrame({
    "Token Sentence":lineList
})

data_output_word2vec = data_output_word2vec.applymap(lambda x: str(x).replace(' ', u"\u00A0"))

data_output_word2vec.to_csv(output_token_for_word2vec, encoding="utf8", sep=" ", header=None, index=None)

100%|██████████| 9362/9362 [00:10<00:00, 934.55it/s] 


# Word2Vec Model Training
* Reference: https://radimrehurek.com/gensim/auto_examples/tutorials/run_word2vec.html#sphx-glr-auto-examples-tutorials-run-word2vec-py
* Reference: https://radimrehurek.com/gensim/auto_examples/tutorials/run_annoy.html

In [11]:
# import packages
from gensim.models import word2vec
from monpa import utils
from tqdm import tqdm
import codecs
import logging
import monpa
import numpy as np
import os
import pandas as pd
import spacy

os.getcwd()

'D:\\Windows_Storage\\Storage\\Github\\KnowledgeGraph\\scripts'

In [12]:
sentences = word2vec.LineSentence(output_token_for_word2vec)

# https://radimrehurek.com/gensim/models/word2vec.html
model = word2vec.Word2Vec(sentences=sentences,
    corpus_file=None,
    size=600,
    alpha=0.025,
    window=5,
    min_count=5,
    max_vocab_size=None,
    sample=0.001,
    seed=1,
    workers=3,
    min_alpha=0.0001,
    sg=0,
    hs=0,
    negative=5,
    ns_exponent=0.75,
    cbow_mean=1,
    iter=1000,
    null_word=0,
    trim_rule=None,
    sorted_vocab=1,
    batch_words=10000,
    compute_loss=False,
    callbacks=(),
    max_final_vocab=None)

model.save(model_save_path)

# Some opertation needed but would make kernel down
from gensim.models import word2vec

model = word2vec.Word2Vec.load("../models/210408_word2vec.model")

print(model.wv.similarity("晶圓", "價值"))
print(model.wv.most_similar(["晶圓", "矽"]))